<a href="https://colab.research.google.com/github/znumunz/TMAO/blob/main/TMAO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**[github](https://github.com/znumunz/TMAO)**

#**Import Library**

In [ ]:
from google.colab import drive
import pandas as pd
pd.set_option('display.max_columns', None)   # แสดงทุกคอลัมน์
# pd.set_option('display.width', None)         # ให้ความกว้างไม่จำกัด
# pd.set_option('display.max_colwidth', None)  # ถ้าค่าในคอลัมน์ยาว ก็แสดงทั้งหมด
import string #ใช้สำหรับการเขียนฟังก์ชัน เลือกคอลัมน์ด้วย ชื่อคอลัมน์ A B C


#**Read Data**

In [ ]:
drive.mount('/content/drive')

In [ ]:
# สร้างทางลัดของ path ไปยัง "My Drive" ก่อน จึงจะสามารถอ่านไฟล์จาก path เดียวกันได้

path = "/content/drive/MyDrive/TMAO/Data/__อันนี้_ล่าสุด_Nov_4_2025__of_EGAT04102024_hs_trop_I_AJVICHAI_11.xlsx"
df = pd.read_excel(path)
df.head(3)

#**Choose columns**

In [ ]:
# เลือกคอลัมน์จากชื่อคอลัมน์
data = df[['empn','case','age','sex']]
data.head()

## ฟังก์ชันการเลือกคอลัมน์แบบใช้ชื่อ A B C:F **ได้ในครั้งเดียว**

In [ ]:
def excel_idx(s):
    n = 0
    for c in s.upper():
        n = n * 26 + (ord(c) - 64)
    return n - 1

def pick(df, *args):
    idx = []
    for a in args:
        if ":" in a:
            s, e = a.split(":")
            idx.extend(range(excel_idx(s), excel_idx(e)+1))
        else:
            idx.append(excel_idx(a))
    return df.iloc[:, idx]


In [ ]:
df = pick(df,"V","Y","W","AC","AO","C:S")
df

#**Clean Data**

In [ ]:
df.info()

In [ ]:
df['dead all cause อันนี้'].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
df[df[['wt', 'height', 'waist', 'hip']].isnull().any(axis=1)]

In [ ]:
for col in ['wt', 'height', 'waist', 'hip']:
    median_val = df[col].median()
    df[col].fillna(median_val, inplace=True)

print("ค่าว่างในคอลัมน์ 'wt', 'height', 'waist', 'hip' หลังจากเติมค่า:")
print(df[['wt', 'height', 'waist', 'hip']].isnull().sum())

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df.isnull().sum()

#**Feature Engineering**

## **Feature Extraction แยกวัน เดือน ปี**

In [ ]:
df

##**คำนวณช่วงเวลาที่มีชีวิต คนที่มีชีวิต สิ้นสุดการรักษา 30/12/2019**

##**แยกกลุ่มเสียชีวิต กับไม่เสียชีวิต**

In [ ]:
df1 = df[df['dead all cause อันนี้']==1]
df1.head(3)

In [ ]:
df0 = df[df['dead all cause อันนี้']==0]
df0.head(3)

#**Data Exploration**

In [ ]:
corre_heatmap = df[['dead all cause อันนี้','age','sex','wt','height','waist','hip','sbpok','dbpok','tc','hdl','ldl','tg','คูณ100','RESULT (ng/L)']]
corre_heatmap.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 10))
sns.heatmap(corre_heatmap.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap of Selected Features')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
sns.boxplot(x='dead all cause อันนี้', y='RESULT (ng/L)', data=df)
plt.title('Distribution of RESULT (ng/L) by dead all cause')
plt.xlabel('Dead All Cause (0: No, 1: Yes)')
plt.ylabel('RESULT (ng/L)')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Filter data for each group
df_dead_0 = df[df['dead all cause อันนี้'] == 0]
df_dead_1 = df[df['dead all cause อันนี้'] == 1]

# Create subplots for histograms
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

# Histogram for dead_all_cause = 0
sns.histplot(df_dead_0['RESULT (ng/L)'], bins=20, kde=True, ax=axes[0], color='skyblue')
axes[0].set_title('RESULT (ng/L) for Dead All Cause = 0')
axes[0].set_xlabel('RESULT (ng/L)')
axes[0].set_ylabel('Frequency')

# Histogram for dead_all_cause = 1
sns.histplot(df_dead_1['RESULT (ng/L)'], bins=20, kde=True, ax=axes[1], color='salmon')
axes[1].set_title('RESULT (ng/L) for Dead All Cause = 1')
axes[1].set_xlabel('RESULT (ng/L)')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
sns.boxplot(x='dead all cause อันนี้', y='คูณ100', data=df)
plt.title('Distribution of CV by dead all caus')
plt.xlabel('Dead All Cause (0: No, 1: Yes)')
plt.ylabel('CV')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Filter data for each group
df_dead_0 = df[df['dead all cause อันนี้'] == 0]
df_dead_1 = df[df['dead all cause อันนี้'] == 1]

# Create subplots for histograms
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)

# Histogram for dead_all_cause = 0
sns.histplot(df_dead_0['คูณ100'], bins=20, kde=True, ax=axes[0], color='skyblue')
axes[0].set_title('CV for Dead All Cause = 0')
axes[0].set_xlabel('CV')
axes[0].set_ylabel('Frequency')

# Histogram for dead_all_cause = 1
sns.histplot(df_dead_1['คูณ100'], bins=20, kde=True, ax=axes[1], color='salmon')
axes[1].set_title('CV for Dead All Cause = 1')
axes[1].set_xlabel('CV')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_dead_1 = df[df['dead all cause อันนี้'] == 1]
df_dead_1['DDATE070809_10y อันนี้'].value_counts()